In [ ]:
#Actualizar version de boto3 y reiniciar kernel
!pip install boto3 botocore aiobotocore --upgrade

In [ ]:
#Instrucciones usando invoke model

In [74]:
import json
import boto3

bedrock = boto3.client(service_name='bedrock-runtime')

# Cambia el model_id para usar diferentes modelos
model_id = 'meta.llama2-13b-chat-v1' 
#model_id = 'mistral.mistral-large-2402-v1:0'
#model_id = 'cohere.command-text-v14'

prompt = "Human: Hola que tal, como estas? "

if model_id.startswith('cohere'):
    input = {'prompt': prompt+" provide your response in english no mather the input language", 'stop_sequences': ["\n"], 'max_tokens': 200,'temperature': 0.5}     # Si el modelo es de Cohere
elif model_id.startswith('mistral'):
    input = {'prompt': prompt.replace("Human", "'role': 'user'"), 'stop': ["\n"], 'temperature': 0.5}     # Si el modelo es de Mistral 
elif model_id.startswith('meta'):
    input = {'prompt': prompt.replace("Human","<s>[INST] {")+" proporciona tu respuesta en espaniol"+"prompt} [/INST]",'max_gen_len': 200,'temperature': 0.5}     # Si el modelo es de Meta 

# Enviar solicitud al modelo seleccionado
body = json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=model_id)
response_body = json.loads(response.get('body').read())

# Imprimir las generaciones para los diferentes modelos
if model_id.startswith('cohere') or model_id.startswith('mistral'):
    completions = response_body.get('generations', response_body.get('outputs', []))
    for part in completions:
        print(part['text'])
else:
    print(response_body['generation'])
    
    #https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModelWithResponseStream_MetaLlama2_section.html
    #https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_CohereCommand_section.html
    #https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_MistralAi_section.html

 Hi! I am very well thank you, how about you?


In [ ]:
#Chat usando invoke model

In [77]:

import json
import boto3

bedrock = boto3.client(service_name='bedrock-runtime')

# Cambia el model_id para usar diferentes modelos
model_id = 'meta.llama2-13b-chat-v1' 
#model_id = 'mistral.mistral-large-2402-v1:0'
#model_id = 'cohere.command-text-v14'

#contexto
prompt = """
Human: Hola que tal, como estas?
AI: Hola, estoy bien, como te encuentras tu?
Human: No puedo encontrar a mi gato.
AI: Lamento escuchar eso. Yo puedo ayudarte a encontrar a tu gato.
Human: Gracias! donde crees que se pueda esconder?
"""


if model_id.startswith('cohere'):
    input = {'prompt': prompt+" provide your response in english no mather the input language", 'stop_sequences': ["\n"], 'max_tokens': 200,'temperature': 0.5}     # Si el modelo es de Cohere
elif model_id.startswith('mistral'):
    input = {'prompt': prompt.replace("Human", "'role': 'user'"), 'stop': ["\n"], 'temperature': 0.5}     # Si el modelo es de Mistral 
elif model_id.startswith('meta'):
    input = {'prompt': prompt.replace("Human","<s>[INST] {")+" proporciona tu respuesta en espaniol"+"prompt} [/INST]",'max_gen_len': 200,'temperature': 0.5}     # Si el modelo es de Meta 

# Enviar solicitud al modelo seleccionado
body = json.dumps(input)
response = bedrock.invoke_model(body=body, modelId=model_id)
response_body = json.loads(response.get('body').read())

# Imprimir las generaciones para los diferentes modelos
if model_id.startswith('cohere') or model_id.startswith('mistral'):
    completions = response_body.get('generations', response_body.get('outputs', []))
    for part in completions:
        print(part['text'])
else:
    print(response_body['generation'])

 I'm sorry, I can only help you in English now. If you're missing your cat, you could try checking the most likely spots where it might hide, such as small spaces, behind furniture, or near its food or water bowl. You could also try calling out its name to see if it responds, or use treats to coax it out of its hiding spot. It's also a good idea to ensure all doors and windows are closed to prevent your cat from accidentally escaping. 



In [ ]:
#instruccion usando bedrock converse api

In [20]:
import json
import boto3

#model_id = 'amazon.titan-text-lite-v1'
#model_id = 'meta.llama2-13b-chat-v1' 
#model_id = 'mistral.mistral-large-2402-v1:0'
bedrock = boto3.client(service_name='bedrock-runtime')
messages = [
    {"role": "user",      "content": [{"text": "Hola, como estas?"}]}
]
inference_config = {"temperature": 0.5, "maxTokens": 200, "stopSequences": []}
response = bedrock.converse(
    modelId=model_id,
    messages=messages,
    inferenceConfig=inference_config
)
content = response['output']['message']['content']
for item in content:
    print(item['text'])

Hello, I'm doing well, thank you. How can I help you today?


In [ ]:
#Chat usando bedrock converse api

In [26]:
import json
import boto3

model_id = 'amazon.titan-text-lite-v1'
#model_id = 'meta.llama2-13b-chat-v1' 
#model_id = 'mistral.mistral-large-2402-v1:0'

bedrock = boto3.client(service_name='bedrock-runtime')
messages = [
    {"role": "user",           "content": [{"text": "Hola, como estas?"}]},
    {"role": "assistant",      "content": [{"text": "Hola, estoy bien, como te encuentras tu?"}]},
    {"role": "user",           "content": [{"text": "No puedo encontrar a mi gato."}]},
    {"role": "assistant",      "content": [{"text": "Lamento escuchar eso. Yo puedo ayudarte a encontrar a tu gato."}]},
    {"role": "user",           "content": [{"text": "Gracias! donde crees que se pueda esconder?"}]},
]

response = bedrock.converse(
    modelId=model_id,
    messages=messages,
    inferenceConfig={"temperature": 0.5, "maxTokens": 200, "stopSequences": []}
)
content = response['output']['message']['content']
for item in content:
    print(item['text'])

Si es un gato callejero, podría estar en un árbol o en un arbusto. También podría estar en un garaje o en un patio trasero.
